In [1]:
import pandas as pd
import numpy as np

# Mainclass analysis

In [2]:
master = pd.read_csv('data/master.csv')
d2d_seen = pd.read_csv('data/final_main_d2d_seen.csv')
d2d_seen_cont = pd.read_csv('data/final_main_d2d_seen_cont.csv')
d2u_seen = pd.read_csv('data/final_main_d2u_seen.csv')
d2u_seen_cont = pd.read_csv('data/final_main_d2u_seen_cont.csv')

In [10]:
d2d_seen.head()

,patent_number,priority_date,cite_mainclass,focal_mainclass,first_seen
0,D466542,1902,D16,D16,1
1,D499408,1904,D14,D14,1
2,D510547,1905,D12,D12,1
3,D523461,1906,D16,D16,0
4,D523461,1906,D14,D16,1


Let's examine how many novel combinations exist in the discrete measure

In [15]:
d2d_seen.loc[d2d_seen.first_seen == 1].groupby('focal_mainclass')['patent_number'].count()

focal_mainclass
D01     21
D02     48
D03     45
D04     44
D05     25
D06     65
D07     55
D08     43
D09     58
D10     43
D11     70
D12     34
D13     43
D14     50
D15     50
D16     36
D17     33
D18     25
D19     67
D20     23
D21     67
D22     18
D23     80
D24    144
D25     70
D26     25
D27     45
D28     34
D29     25
D30     29
D32     43
D34     29
D99     41
Name: patent_number, dtype: int64

In [16]:
d2u_seen.loc[d2u_seen.first_seen == 1].groupby('focal_mainclass')['patent_number'].count()

focal_mainclass
D01    305
D02    444
D03    510
D04    349
D05    411
D06    673
D07    532
D08    751
D09    575
D10    593
D11    495
D12    539
D13    632
D14    614
D15    589
D16    451
D17    258
D18    450
D19    564
D20    461
D21    543
D22    457
D23    690
D24    819
D25    495
D26    538
D27    374
D28    416
D29    396
D30    408
D32    450
D34    491
D99    324
Name: patent_number, dtype: int64

In [27]:
print('Design to design number:',d2d_seen.loc[d2d_seen.first_seen == 1].patent_number.nunique(),'\n',
      'Design to design percentage of dataset', d2d_seen.loc[d2d_seen.first_seen == 1].patent_number.nunique()/master.patent_number.nunique(), '\n',
     'Design to utility number:',d2u_seen.loc[d2u_seen.first_seen == 1].patent_number.nunique(),'\n',
      'Design to utility percentage of dataset', d2u_seen.loc[d2u_seen.first_seen == 1].patent_number.nunique()/master.patent_number.nunique())

Design to design number: 1026 
 Design to design percentage of dataset 0.0019524634150982893 
 Design to utility number: 10496 
 Design to utility percentage of dataset 0.019973738796171193


Examin all design patents together to see how many patents are novel in total

In [28]:
all_seen = pd.concat([d2d_seen, d2u_seen])
print('all novel design patents:', all_seen.loc[all_seen.first_seen == 1].patent_number.nunique())
print('percent of dataset novel:', all_seen.loc[all_seen.first_seen == 1].patent_number.nunique()/master.patent_number.nunique())

all novel design patents: 11175
percent of dataset novel: 0.021265866143979906


means table for each focal design catagory

In [31]:
d2d_seen.groupby('focal_mainclass')['first_seen'].describe()

,count,mean,std,min,25%,50%,75%,max
focal_mainclass,,,,,,,,
D01,6593.0,0.003185,0.056352,0.0,0.0,0.0,0.0,1.0
D02,27623.0,0.001738,0.041650,0.0,0.0,0.0,0.0,1.0
D03,38424.0,0.001171,0.034202,0.0,0.0,0.0,0.0,1.0
D04,7819.0,0.005627,0.074809,0.0,0.0,0.0,0.0,1.0
D05,5015.0,0.004985,0.070436,0.0,0.0,0.0,0.0,1.0
D06,79474.0,0.000818,0.028587,0.0,0.0,0.0,0.0,1.0
D07,59429.0,0.000925,0.030408,0.0,0.0,0.0,0.0,1.0
D08,59637.0,0.000721,0.026843,0.0,0.0,0.0,0.0,1.0
D09,67506.0,0.000859,0.029299,0.0,0.0,0.0,0.0,1.0


In [32]:
d2u_seen.groupby('focal_mainclass')['first_seen'].describe()

,count,mean,std,min,25%,50%,75%,max
focal_mainclass,,,,,,,,
D01,4944.0,0.061691,0.240618,0.0,0.0,0.0,0.0,1.0
D02,31882.0,0.013926,0.117187,0.0,0.0,0.0,0.0,1.0
D03,52616.0,0.009693,0.097975,0.0,0.0,0.0,0.0,1.0
D04,12818.0,0.027227,0.162752,0.0,0.0,0.0,0.0,1.0
D05,8260.0,0.049758,0.217457,0.0,0.0,0.0,0.0,1.0
D06,86547.0,0.007776,0.087839,0.0,0.0,0.0,0.0,1.0
D07,72832.0,0.007304,0.085154,0.0,0.0,0.0,0.0,1.0
D08,102477.0,0.007328,0.085293,0.0,0.0,0.0,0.0,1.0
D09,84551.0,0.006801,0.082186,0.0,0.0,0.0,0.0,1.0
